In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_selection import SelectKBest, chi2
from cProfile import label
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder
#import torch
#import torch.nn as nn
#import torch.optim as optim
#from torch.utils.data import DataLoader, TensorDataset


In [2]:
df_train = pd.read_csv("Dataset/diabetic_data_training.csv")
df_test = pd.read_csv('Dataset/diabetic_data_test.csv')

display(df_train.shape, df_test.shape)

(91589, 50)

(10177, 50)

In [3]:
mapping_csv = pd.read_csv("Dataset/IDS_mapping.csv")
mapping_csv = mapping_csv.dropna(how='all')
dictionaries = {}
current_dict = 'admission_type_id'
dictionaries[current_dict] = {}


for index, row in mapping_csv.iterrows():
   id = row.to_list()[0]
   if id.isdigit():
       dictionaries[current_dict][row.to_list()[0]] = str(row.to_list()[1])
   else:
       current_dict = id
       dictionaries[current_dict] = {}

In [4]:
# Mapping the textual values associated with teh placeholder values given originally in teh dataset

# TRAIN DATA

if 'admission_type_id' in df_train.columns:
   df_train['admission_type_id'] = df_train['admission_type_id'].astype(str).map(dictionaries['admission_type_id'])


# Map discharge dispositions
if 'discharge_disposition_id' in df_train.columns:
   df_train['discharge_disposition_id'] = df_train['discharge_disposition_id'].astype(str).map(dictionaries['discharge_disposition_id'])


if 'admission_source_id' in df_train.columns:
   df_train['admission_source_id'] = df_train['admission_source_id'].astype(str).map(dictionaries['admission_source_id'])
   
# TEST DATA

if 'admission_type_id' in df_test.columns:
   df_test['admission_type_id'] = df_test['admission_type_id'].astype(str).map(dictionaries['admission_type_id'])


# Map discharge dispositions
if 'discharge_disposition_id' in df_test.columns:
   df_test['discharge_disposition_id'] = df_test['discharge_disposition_id'].astype(str).map(dictionaries['discharge_disposition_id'])


if 'admission_source_id' in df_test.columns:
   df_test['admission_source_id'] = df_test['admission_source_id'].astype(str).map(dictionaries['admission_source_id'])
   
display(df_train.head(10), df_test.head(10), df_train.shape, df_test.shape)


,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,149190,55629189,Caucasian,Female,[10-20),?,Emergency,Discharged to home,Emergency Room,3,...,No,Up,No,No,No,No,No,Ch,Yes,>30
1,64410,86047875,AfricanAmerican,Female,[20-30),?,Emergency,Discharged to home,Emergency Room,2,...,No,No,No,No,No,No,No,No,Yes,NO
2,500364,82442376,Caucasian,Male,[30-40),?,Emergency,Discharged to home,Emergency Room,2,...,No,Up,No,No,No,No,No,Ch,Yes,NO
3,16680,42519267,Caucasian,Male,[40-50),?,Emergency,Discharged to home,Emergency Room,1,...,No,Steady,No,No,No,No,No,Ch,Yes,NO
4,35754,82637451,Caucasian,Male,[50-60),?,Urgent,Discharged to home,Clinic Referral,3,...,No,Steady,No,No,No,No,No,No,Yes,>30
5,55842,84259809,Caucasian,Male,[60-70),?,Elective,Discharged to home,Clinic Referral,4,...,No,Steady,No,No,No,No,No,Ch,Yes,NO
6,63768,114882984,Caucasian,Male,[70-80),?,Emergency,Discharged to home,Emergency Room,5,...,No,No,No,No,No,No,No,No,Yes,>30
7,12522,48330783,Caucasian,Female,[80-90),?,Urgent,Discharged to home,Transfer from a hospital,13,...,No,Steady,No,No,No,No,No,Ch,Yes,NO
8,15738,63555939,Caucasian,Female,[90-100),?,Elective,Discharged/transferred to SNF,Transfer from a hospital,12,...,No,Steady,No,No,No,No,No,Ch,Yes,NO
9,36900,77391171,AfricanAmerican,Male,[60-70),?,Urgent,Discharged to home,Transfer from a hospital,7,...,No,Steady,No,No,No,No,No,Ch,Yes,<30


,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),?,nan,Not Mapped,Physician Referral,1,...,No,No,No,No,No,No,No,No,No,NO
1,28236,89869032,AfricanAmerican,Female,[40-50),?,Emergency,Discharged to home,Emergency Room,9,...,No,Steady,No,No,No,No,No,No,Yes,>30
2,150006,22864131,?,Female,[50-60),?,Urgent,Discharged to home,Transfer from a hospital,2,...,No,Down,No,No,No,No,No,Ch,Yes,NO
3,253380,56480238,AfricanAmerican,Female,[60-70),?,Emergency,Discharged to home,Emergency Room,6,...,No,Steady,No,No,No,No,No,Ch,Yes,NO
4,383430,80588529,Caucasian,Female,[70-80),?,Emergency,Discharged/transferred to another short term h...,Emergency Room,1,...,No,Down,No,No,No,No,No,Ch,Yes,>30
5,550098,21820806,AfricanAmerican,Male,[50-60),?,Urgent,Discharged to home,Clinic Referral,4,...,No,No,No,No,No,No,No,No,No,<30
6,676422,63754317,AfricanAmerican,Female,[70-80),?,Emergency,Discharged to home,Emergency Room,4,...,No,Steady,No,No,No,No,No,No,Yes,>30
7,870294,95075649,Caucasian,Female,[70-80),?,Emergency,Discharged/transferred to home with home healt...,Emergency Room,7,...,No,No,No,No,No,No,No,No,No,<30
8,1072554,114039603,Caucasian,Male,[70-80),?,Elective,Discharged to home,Clinic Referral,3,...,No,Steady,No,No,No,No,No,Ch,Yes,NO
9,1161024,20830941,Caucasian,Female,[70-80),?,Emergency,Discharged/transferred to another short term h...,Emergency Room,5,...,No,Steady,No,No,No,No,No,Ch,Yes,>30


(91589, 50)

(10177, 50)

In [5]:
# Create df_train_ and df_test_ dataframes, which are simply dropping the columns below
features_to_drop = ['payer_code', 'medical_specialty', 'weight'] # remove 'encounter_id', 'patient_nbr' to keep temporal information

# Drop columns directly from both dataframes
df_train_ = df_train.drop(columns=features_to_drop, axis=1)
df_test_ = df_test.drop(columns=features_to_drop, axis=1)


In [6]:
# Generate Timestep for Each Patient (Visit Number)
df_train_['timestep'] = df_train_.groupby('patient_nbr').cumcount() + 1
df_test_['timestep'] = df_test_.groupby('patient_nbr').cumcount() + 1

# Reorder columns to place timestep as the 3rd column train dataset
columns = list(df_train_.columns)
columns.insert(2, columns.pop(columns.index('timestep')))
df_train_ = df_train_[columns]

# Select Relevant Columns (Keep Timestep)
temporal_cols = [col for col in df_train_.columns]# if col not in ['readmitted']]
df_train_ = df_train_[temporal_cols]


# Reorder columns to place timestep as the 3rd column test dataset
columns = list(df_test_.columns)
columns.insert(2, columns.pop(columns.index('timestep')))
df_test_ = df_test_[columns]

In [7]:
# Replace '?' with NaN
df_train_ = df_train_.replace('?', pd.NA)
df_test_ = df_test_.replace('?', pd.NA)

# Replace None or 'none' values in the specified columns with the desired text
df_train_['max_glu_serum'] = df_train_['max_glu_serum'].replace(
    [None, 'none'], "The test was not conducted on this patient"
)
df_train_['A1Cresult'] = df_train_['A1Cresult'].replace(
    [None, 'none'], "The test was not conducted on this patient"
)

# Replace None or 'none' values in the specified columns with the desired text
df_test_['max_glu_serum'] = df_test_['max_glu_serum'].replace(
    [None, 'none'], "The test was not conducted on this patient"
)
df_test_['A1Cresult'] = df_test_['A1Cresult'].replace(
    [None, 'none'], "The test was not conducted on this patient"
)


In [8]:
# Checking for Missing Values and Printing Only Features with Missing Values
missing_values_df_train_ = df_train_.isnull().sum()
# Filter for features with missing values
missing_values_df_train_ = missing_values_df_train_[missing_values_df_train_ > 0]

missing_values_df_test_ = df_test_.isnull().sum()
# Filter for features with missing values
missing_values_df_test_ = missing_values_df_test_[missing_values_df_test_ > 0]

print("\nFeatures with Missing Values in df_train_:")
print(missing_values_df_train_)



Features with Missing Values in df_train_:
race      2059
diag_1      21
diag_2     320
diag_3    1266
dtype: int64


In [9]:
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np

# First, let's handle categorical variables
# Create label encoders
label_encoders = {}
other_cols = []
columns_to_impute = ['race', 'diag_1', 'diag_2', 'diag_3']

# Function to encode and decode 
def encode_categorical(df, columns):
    df_encoded = df.copy()
    for col in columns:
        if df[col].dtype == 'object':  # If column is categorical
            if col not in label_encoders:
                label_encoders[col] = LabelEncoder()
                # Fill NaN with a placeholder first
                df_encoded[col] = df_encoded[col].fillna('Missing')
                # Fit and transform
                df_encoded[col] = label_encoders[col].fit_transform(df_encoded[col])
    return df_encoded

def decode_categorical(df):
    df_decoded = df.copy()
    for col in columns_to_impute:
        if col in label_encoders:
            df_decoded[col] = label_encoders[col].inverse_transform(df_decoded[col].astype(int))
    return df_decoded

# Encode both training and test data
df_train_encoded = encode_categorical(df_train_, columns_to_impute)

# Now apply KNN imputation
imputer = KNNImputer(n_neighbors=5)

# Fit and transform training data
df_train_imputed = pd.DataFrame(
    imputer.fit_transform(df_train_encoded[columns_to_impute]),
    columns=columns_to_impute,
    index=df_train_encoded.index
)

# Round the values since we're working with categorical data
for col in columns_to_impute:
    df_train_imputed[col] = df_train_imputed[col].round()

# Decode the imputed values back to categories
df_train_imputed_decoded = decode_categorical(df_train_imputed)

for col in df_train_:
    if col not in columns_to_impute:
        other_cols.append(col)
if other_cols:
    df_train_ = pd.concat([df_train_imputed_decoded, df_train_[other_cols]], axis=1)


In [10]:
# convert age from categorical to numerical variable
age_mapping = {
       '[0-10)' : 5,
       '[10-20)' : 15,
       '[20-30)' : 25,
       '[30-40)' : 35,
       '[40-50)' : 45,
       '[50-60)' : 55,
       '[60-70)' : 65,
       '[70-80)' : 75,
       '[80-90)' : 85,
       '[90-100)' : 95,
}

df_train_['age'] = df_train_['age'].map(age_mapping)
df_test_['age'] = df_test_['age'].map(age_mapping)



In [11]:
df_train_ = df_train_.dropna()
df_test_ = df_test_.dropna()
print(df_train_.shape, df_test_.shape)

(91589, 48) (9810, 48)


In [12]:
readmitted_col_df_train_ = df_train_['readmitted']
df_train_copy = df_train_.drop(columns=['readmitted'], axis=1)

readmitted_col_df_test_ = df_test_['readmitted']
df_test_copy = df_test_.drop(columns=['readmitted'], axis=1)

####################################

# create a list with the categorical columns and one with the numerical columns
categorical_columns = df_train_copy.select_dtypes(include=['object']).columns.tolist()
numerical_columns = df_train_copy.select_dtypes(include=['int64', 'float64']).columns.tolist()


In [13]:
df_train_one = pd.get_dummies(df_train_[categorical_columns],drop_first=False) 
df_test_one = pd.get_dummies(df_test_[categorical_columns],drop_first=False)


In [14]:
# Select numerical columns for df_train
df_train_num = df_train_[numerical_columns]
# Select numerical columns for df_test
df_test_num = df_test_[numerical_columns]

In [15]:
df_train_seq = pd.concat([df_train_num, df_train_one, readmitted_col_df_train_], axis = 1)
df_test_seq = pd.concat([df_test_num, df_test_one, readmitted_col_df_test_], axis = 1)

print(df_train_seq.shape)
print(df_test_seq.shape)

display(df_train_seq,df_test_seq)

(91589, 2371)
(9810, 1527)


,encounter_id,patient_nbr,timestep,age,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,...,glimepiride-pioglitazone_No,metformin-rosiglitazone_No,metformin-rosiglitazone_Steady,metformin-pioglitazone_No,metformin-pioglitazone_Steady,change_Ch,change_No,diabetesMed_No,diabetesMed_Yes,readmitted
0,149190,55629189,1,15,3,59,0,18,0,0,...,True,True,False,True,False,True,False,False,True,>30
1,64410,86047875,1,25,2,11,5,13,2,0,...,True,True,False,True,False,False,True,False,True,NO
2,500364,82442376,1,35,2,44,1,16,0,0,...,True,True,False,True,False,True,False,False,True,NO
3,16680,42519267,1,45,1,51,0,8,0,0,...,True,True,False,True,False,True,False,False,True,NO
4,35754,82637451,1,55,3,31,6,16,0,0,...,True,True,False,True,False,False,True,False,True,>30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91584,443847548,100162476,2,75,3,51,0,16,0,0,...,True,True,False,True,False,True,False,False,True,>30
91585,443847782,74694222,1,85,5,33,3,18,0,0,...,True,True,False,True,False,False,True,False,True,NO
91586,443854148,41088789,3,75,1,53,0,9,1,0,...,True,True,False,True,False,True,False,False,True,NO
91587,443857166,31693671,4,85,10,45,2,21,0,0,...,True,True,False,True,False,True,False,False,True,NO


,encounter_id,patient_nbr,timestep,age,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,...,glipizide-metformin_No,glimepiride-pioglitazone_No,glimepiride-pioglitazone_Steady,metformin-rosiglitazone_No,metformin-pioglitazone_No,change_Ch,change_No,diabetesMed_No,diabetesMed_Yes,readmitted
1,28236,89869032,1,45,9,47,2,17,0,0,...,True,True,False,True,True,False,True,False,True,>30
3,253380,56480238,1,65,6,87,0,18,0,0,...,True,True,False,True,True,True,False,False,True,NO
4,383430,80588529,1,75,1,28,0,15,0,0,...,True,True,False,True,True,True,False,False,True,>30
5,550098,21820806,1,55,4,40,1,14,0,0,...,True,True,False,True,True,False,True,True,False,<30
6,676422,63754317,1,75,4,48,2,15,0,1,...,True,True,False,True,True,False,True,False,True,>30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10172,443266892,180189752,1,55,5,71,5,23,0,0,...,True,True,False,True,True,True,False,False,True,>30
10173,443721236,38726739,1,75,3,57,4,11,0,0,...,True,True,False,True,True,False,True,True,False,NO
10174,443778398,134647673,1,45,1,1,5,8,0,0,...,True,True,False,True,True,True,False,False,True,NO
10175,443824292,138784172,2,85,3,31,2,24,0,0,...,True,True,False,True,True,True,False,False,True,<30


In [16]:
df_train_seq = df_train_seq.replace({'Yes': 1, 'No': 0, True: 1, False: 0})
df_test_seq= df_test_seq.replace({"yes":1, "No":0, True:1, False:0})

display(df_train_seq.head(), df_test_seq.head())

C:\Users\vidur\AppData\Local\Temp\ipykernel_37912\1734839941.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_train_seq = df_train_seq.replace({'Yes': 1, 'No': 0, True: 1, False: 0})
C:\Users\vidur\AppData\Local\Temp\ipykernel_37912\1734839941.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_test_seq= df_test_seq.replace({"yes":1, "No":0, True:1, False:0})


,encounter_id,patient_nbr,timestep,age,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,...,glimepiride-pioglitazone_No,metformin-rosiglitazone_No,metformin-rosiglitazone_Steady,metformin-pioglitazone_No,metformin-pioglitazone_Steady,change_Ch,change_No,diabetesMed_No,diabetesMed_Yes,readmitted
0,149190,55629189,1,15,3,59,0,18,0,0,...,1,1,0,1,0,1,0,0,1,>30
1,64410,86047875,1,25,2,11,5,13,2,0,...,1,1,0,1,0,0,1,0,1,NO
2,500364,82442376,1,35,2,44,1,16,0,0,...,1,1,0,1,0,1,0,0,1,NO
3,16680,42519267,1,45,1,51,0,8,0,0,...,1,1,0,1,0,1,0,0,1,NO
4,35754,82637451,1,55,3,31,6,16,0,0,...,1,1,0,1,0,0,1,0,1,>30


,encounter_id,patient_nbr,timestep,age,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,...,glipizide-metformin_No,glimepiride-pioglitazone_No,glimepiride-pioglitazone_Steady,metformin-rosiglitazone_No,metformin-pioglitazone_No,change_Ch,change_No,diabetesMed_No,diabetesMed_Yes,readmitted
1,28236,89869032,1,45,9,47,2,17,0,0,...,1,1,0,1,1,0,1,0,1,>30
3,253380,56480238,1,65,6,87,0,18,0,0,...,1,1,0,1,1,1,0,0,1,NO
4,383430,80588529,1,75,1,28,0,15,0,0,...,1,1,0,1,1,1,0,0,1,>30
5,550098,21820806,1,55,4,40,1,14,0,0,...,1,1,0,1,1,0,1,1,0,<30
6,676422,63754317,1,75,4,48,2,15,0,1,...,1,1,0,1,1,0,1,0,1,>30


In [17]:
# # Sort by 'patient_nbr' and 'timestep' to ensure proper cumulative calculations
df_train_seq = df_train_seq.sort_values(by=['patient_nbr', 'timestep'])
df_test_seq = df_test_seq.sort_values(by=['patient_nbr', 'timestep'])

# Step 4: Fill NaN Values for the First Visit (No History)
df_train_seq = df_train_seq.fillna(0)
df_test_seq = df_test_seq.fillna(0)

# Display the updated DataFrame
print(df_train_seq.shape)
print(df_test_seq.shape)

display(df_train_seq)


(91589, 2371)
(9810, 1527)


,encounter_id,patient_nbr,timestep,age,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,...,glimepiride-pioglitazone_No,metformin-rosiglitazone_No,metformin-rosiglitazone_Steady,metformin-pioglitazone_No,metformin-pioglitazone_Steady,change_Ch,change_No,diabetesMed_No,diabetesMed_Yes,readmitted
3840,24437208,135,1,55,8,77,6,33,0,0,...,1,1,0,1,0,1,0,0,1,<30
5244,29758806,378,1,55,2,49,1,11,0,0,...,1,1,0,1,0,0,1,1,0,NO
60847,189899286,729,1,85,4,68,2,23,0,0,...,1,1,0,1,0,0,1,0,1,NO
15744,64331490,774,1,85,3,46,0,20,0,0,...,1,1,0,1,0,1,0,0,1,NO
1047,8380170,1152,1,55,6,43,2,13,0,0,...,1,1,0,1,0,0,1,0,1,>30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89876,418513058,189351095,1,85,1,73,1,11,0,0,...,1,1,0,1,0,0,1,1,0,NO
85753,359719064,189365864,1,65,3,56,1,8,0,0,...,1,1,0,1,0,0,1,0,1,NO
84285,338462954,189445127,1,85,3,39,0,18,0,0,...,1,1,0,1,0,1,0,0,1,NO
91573,443811536,189481478,1,45,14,69,0,16,0,0,...,1,1,0,1,0,1,0,0,1,>30


In [18]:
# Mapping readmitted column to numerical values
label_encoder = LabelEncoder()

label_encoder.fit_transform(readmitted_col_df_train_)
df_train_target = label_encoder.transform(readmitted_col_df_train_)

label_encoder.fit_transform(readmitted_col_df_test_)
df_test_target = label_encoder.transform(readmitted_col_df_test_)

class_mapping = {label: index for index, label in enumerate(label_encoder.classes_)}
print("Mapping of classes to encoded values:", class_mapping)

Mapping of classes to encoded values: {'<30': 0, '>30': 1, 'NO': 2}


In [ ]:
df_train_chi_seq = df_train_seq.drop(columns=['readmitted'], axis=1)
df_test_chi_seq = df_test_seq.drop(columns=['readmitted'], axis=1)

# Step 1: Group by Patient ID (Assuming `patient_nbr` is the unique identifier)
grouped = df_train_chi_seq.groupby('patient_nbr')

df_train_chi_seq['readmitted_target'] = df_train_target
# Count the number of rows (visits) for each patient
sequence_lengths = grouped.size()

# Determine the maximum sequence length
max_seq_length = sequence_lengths.max()

print("Maximum sequence length:", max_seq_length)

# Step 2: Determine the Maximum Sequence Length
#max_seq_length = 5  # Set this based on your dataset (e.g., the maximum number of visits per patient)

# Step 3: Create Sequences
sequences = []
labels = []

for patient_id, group in grouped:
    # Sort rows by time if necessary (e.g., admission date)
    group = group.sort_values(by='time_in_hospital')

    # Extract features and target
    features = group.drop(columns=['patient_nbr', 'readmitted_target']).values
    #target = group['readmitted'].iloc[0]  # Use the target for the first admission as the label
    target = group['readmitted_target'].iloc[0]  # Use the target for the first admission as the label

    # Truncate or Pad to `max_seq_length`
    if len(features) > max_seq_length:
        features = features[:max_seq_length]
    else:
        features = np.pad(features, ((0, max_seq_length - len(features)), (0, 0)), mode='constant')

    # Append the sequence and label
    sequences.append(features)
    labels.append(target)

# Convert to NumPy arrays
X = np.array(sequences, dtype=np.float32)  # Shape: (number_of_sequences, sequence_length, number_of_features)
y = np.array(labels, dtype=np.int64)       # Shape: (number_of_sequences,)


Maximum sequence length: 39


In [20]:
def preprocess_and_select_features(df_train, df_test, target_column, desired_contribution=0.99):
    # Step 1: Split Dataset into Features and Target
    y_train = df_train[target_column]
    X_train = df_train.drop(columns=[target_column])
    y_test = df_test[target_column]
    X_test = df_test.drop(columns=[target_column])

    # Align train and test columns
    X_train, X_test = X_train.align(X_test, join="inner", axis=1)

    # Ensure the target variable is numeric
    label_encoder = LabelEncoder()
    y_train = label_encoder.fit_transform(y_train)
    y_test = label_encoder.transform(y_test)

    # Retain column names for later use
    feature_columns = X_train.columns

    # Step 2: Scale Features using MinMaxScaler
    minmax_scaler = MinMaxScaler()
    X_train_scaled = minmax_scaler.fit_transform(X_train)
    X_test_scaled = minmax_scaler.transform(X_test)

    # Step 3: Apply Chi-Square Test
    print("Applying Chi-Square Test...")
    chi2_selector = SelectKBest(score_func=chi2, k='all')  # Use 'all' to get scores for all features
    chi2_selector.fit(X_train_scaled, y_train)

    # Chi-Square scores and cumulative explained contribution
    chi2_scores = pd.Series(chi2_selector.scores_, index=feature_columns).sort_values(ascending=False)
    cumulative_scores_chi2 = np.cumsum(chi2_scores) / chi2_scores.sum()

    # Step 4: Determine Optimal Number of Features
    optimal_k_chi2 = np.argmax(cumulative_scores_chi2 >= desired_contribution) + 1
    print(f"Optimal number of features to retain from Chi-Square: {optimal_k_chi2}")

    # Step 5: Select Top Features
    top_features_chi2 = chi2_scores.head(optimal_k_chi2).index.tolist()

    # Handle case when no features meet the threshold
    if len(top_features_chi2) == 0:
        print("No features selected based on Chi-Square test. Retaining all features.")
        top_features_chi2 = feature_columns.tolist()

    # Reduce datasets to Chi-Square selected features
    X_train_selected = pd.DataFrame(X_train_scaled, columns=feature_columns)[top_features_chi2]
    X_test_selected = pd.DataFrame(X_test_scaled, columns=feature_columns)[top_features_chi2]

    return X_train_selected, X_test_selected, y_train, y_test, top_features_chi2

# Apply the function to train and test datasets
X_train_selected, X_test_selected, y_train, y_test, selected_features = preprocess_and_select_features(
    df_train_seq, df_test_seq, target_column="readmitted"
)

# Output the selected features
print("Selected Features:", selected_features)
print("Train Data Shape:", X_train_selected.shape)


Applying Chi-Square Test...
Optimal number of features to retain from Chi-Square: 1119
Selected Features: ['discharge_disposition_id_Expired', 'number_inpatient', 'discharge_disposition_id_Discharged/transferred to another rehab fac including rehab units of a hospital .', 'diag_1_428', 'discharge_disposition_id_Discharged/transferred to home with home health service', 'admission_source_id_ Transfer from another health care facility', 'timestep', 'admission_source_id_Transfer from a hospital', 'diag_2_250', 'diabetesMed_No', 'diag_2_403', 'admission_source_id_ Emergency Room', 'admission_type_id_Elective', 'diag_3_401', 'insulin_Down', 'discharge_disposition_id_Hospice / medical facility', 'diag_3_250', 'diag_1_V58', 'discharge_disposition_id_Discharged to home', 'diag_2_401', 'diag_3_585', 'diag_1_491', 'discharge_disposition_id_Discharged/transferred to SNF', 'discharge_disposition_id_Hospice / home', 'admission_type_id_nan', 'diag_3_403', 'insulin_Up', 'diag_1_250.6', 'insulin_No', '